# [M_04] PROTOCOL: LONG-TERM MEMORY (QDRANT RAG)

**PROJECT:** OMNI-OPERATOR-V1  
**ENGINE:** GEMINI 3 FLASH + TEXT-EMBEDDING-004  
**STATUS:** RECOGNITION_IMPLEMENTATION

This module is responsible for building the system's "strategic memory". We save analysis results (M_01) and generated posts (M_02) in **Qdrant** vector database.

**Why are we doing this?**
1. **Style Analysis:** System can check what hooks it used in the past.
2. **Avoiding Repetition:** Agent knows what it already wrote about, to avoid duplicating content.
3. **Sovereign Data:** Your knowledge about what "works" stays on your server in Docker.

In [1]:
import os
import sys
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from google import genai

# 1. WORKING DIRECTORY CORRECTION
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Adding src to path 
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings

# 2. CLIENT INITIALIZATION
# Qdrant is running on port 6333 in Docker
qclient = QdrantClient(url=settings.qdrant_url, timeout=60)

# Google configuration for embeddings (converting text to vectors)
client = genai.Client(api_key=settings.gemini_api_key)

print(f"LOG: Memory system ready. Connected to Qdrant at {settings.qdrant_url}")

LOG: Memory system ready. Connected to Qdrant at http://localhost:6333


## 1. Creating a Collection (Vector Database Schema)

We define the `content_memory` collection. The vector size (768) corresponds to the latest Google `text-embedding-004` model.

In [2]:
COLLECTION_NAME = "content_memory"

def init_memory():
    """Creates a collection in Qdrant if it doesn't already exist."""
    collections = qclient.get_collections().collections
    exists = any(c.name == COLLECTION_NAME for c in collections)
    
    if not exists:
        print(f"LOG: Creating new collection: {COLLECTION_NAME}")
        qclient.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=768, distance=Distance.COSINE),
        )
        print("✅ COLLECTION CREATED")
    else:
        print(f"LOG: Collection {COLLECTION_NAME} already exists.")

init_memory()

LOG: Collection content_memory already exists.


## 2. Embedding Function

We convert human text into a list of numbers (a vector) that AI can compare mathematically.

In [3]:
def get_embedding(text: str):
    result = client.models.embed_content(
        model="text-embedding-004",
        contents=text
    )
    return result.embeddings[0].values

# TEST:
sample_vec = get_embedding("AI Engineering at Kuźnia Operatorów")
print(f"LOG: Generated vector length: {len(sample_vec)}")

LOG: Generated vector length: 768


## 3. Saving Campaigns to Memory

Implementing the "memorization" logic. We save the campaign content along with its metadata (topic, platforms).

In [4]:
from datetime import datetime

def save_campaign_to_memory(brief_data: dict, topic: str):
    """Saves the campaign report to the Qdrant database, including clip metadata."""
    
    print(f"LOG: Generating embedding for topic: {topic}...")
    # 1. Retrieve the vector (based on the topic and general strategy)
    content_to_embed = f"Topic: {topic}. Strategy: {brief_data['overall_strategy']}"
    vector = get_embedding(content_to_embed)
    
    # 2. Prepare the data point
    point_id = str(uuid.uuid4())
    timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Extraction of clip metadata (durations and hooks)
    clips_meta = [
        {
            "idx": c.get('clip_index'),
            "duration": c.get('duration_seconds'),
            "hook_sample": c.get('posts', [{}])[0].get('content', '')[:50]
        } for c in brief_data.get('clip_strategies', [])
    ]
    
    print(f"LOG: Sending to Qdrant (ID: {point_id}) with clip metadata...")
    
    # 3. Save the point to the database
    qclient.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            PointStruct(
                id=point_id,
                vector=vector,
                payload={
                    "topic": topic,
                    "strategy": brief_data['overall_strategy'],
                    "clips": clips_meta, # <--- This is where our new knowledge about duration goes
                    "type": "campaign_brief",
                    "timestamp": timestamp_str
                }
            )
        ]
    )
    print(f"✅ SUCCESSFULLY REMEMBERED: {topic} ({len(clips_meta)} clips)")

# TEST COMPLIANT WITH THE NEW STANDARD:
mock_brief = {
    "overall_strategy": "Positioning Takzen Dev as a leader in sovereign AI.",
    "clip_strategies": [
        {"clip_index": 1, "duration_seconds": 25, "posts": [{"content": "Eliminating SaaS..."}]},
        {"clip_index": 2, "duration_seconds": 45, "posts": [{"content": "Private Docker..."}]}
    ]
}
save_campaign_to_memory(mock_brief, "AI Sovereignty 2026")

LOG: Generating embedding for topic: AI Sovereignty 2026...
LOG: Sending to Qdrant (ID: af02bb5a-aa3a-4411-87bd-2df9e418e559) with clip metadata...
✅ SUCCESSFULLY REMEMBERED: AI Sovereignty 2026 (2 clips)


## 4. Semantic Search (RAG Test)

We are checking if the system can find related content without using keywords, based solely on the "meaning" of the query.

In [5]:
def search_memory(query: str, limit: int = 2):
    """
    Searches memory using modern Qdrant API 1.16+.
    """
    print(f"LOG: Generating embedding for query: '{query}'...")
    query_vector = get_embedding(query)
    
    print("LOG: Searching vector database...")
    
    # Using the latest query_points API (2025/2026 Standard)
    hits = qclient.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=limit,
        with_payload=True
    ).points
    
    if not hits:
        print("ℹ️ No matching memories found in the database.")
        return

    print(f"\n🔎 RESULTS FOR: '{query}'")
    print("-" * 40)
    for hit in hits:
        score = hit.score
        topic = hit.payload.get('topic', 'No topic')
        strategy = hit.payload.get('strategy', '')[:100]
        print(f" -> [Match: {score:.2f}] {topic}")
        print(f"    Strategy: {strategy}...\n")

# OPERATIONAL TEST:
search_memory("Looking for something about SaaS independence")

LOG: Generating embedding for query: 'Looking for something about SaaS independence'...
LOG: Searching vector database...

🔎 RESULTS FOR: 'Looking for something about SaaS independence'
----------------------------------------
 -> [Match: 0.38] AI Sovereignty 2026
    Strategy: Positioning Takzen Dev as a leader in sovereign AI....



## STATUS: MODULE 04 COMPLETED

We have a fully functional vector memory. Your system is now capable of gathering experience.

**Achievements:**
1. Integration of local Qdrant with Google Embeddings.
2. Ability to store and retrieve knowledge (RAG).
3. Foundations for Phase 5 (Conductor), where the system will check the database before every edit/assembly.